In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import dcg_score, ndcg_score
from collections import OrderedDict
from tqdm import tqdm
tqdm.pandas()
from functools import partial

# **Welcome to week 4 project!**

Welcome to the last week of the course -- so excited to see that you've made it to the end! 👏 

We’ve already discussed the importance of measuring model performance. As Lord Kelvin said, “To measure is to know – If you cannot measure it, you cannot improve it.” And he was right – metrics are the only way we can actually evaluate our model’s performance!

In this week's project, we will touch upon two key aspects related to evaluation:
1. Behavioral metrics
2. Off-policy evaluation

For behavioral metrics, we will work with Spotify music sessions dataset, and implement a few behavioral metrics and understand their relationships with traditional metrics.

For off-policy evaluation, we will simulate a dataset where we have logged action policies, and see how IPS is implemented.


### Goals for this week's project:

For this week's project assignment, we will complete the following tasks:
1. Implement 3 behavioral metrics and present the correlation plot for them
2. Implement another ranking logic (e.g. sort by track popularity, or sort by danceability and compare all metrics on productional ranking logic and this new ranking logic.
3. Complete the implementation of two off-policy estimators: Capped IPS and Normalized Capped Importance Sampling (NCIS)


# Part A: Behavioral metrics

Behavioral metrics include factors like what items a user interacts with and how, the amount of time they spend on the platform, and how they spend that time.

To define and implement a few behavioral metrics, we will work with the Spotify music sessions dataset.
Download the dataset from GDrive: https://drive.google.com/drive/folders/10LGZMgXRuz2qPr_QDbYdVVlKEcnQ25YL?usp=sharing
(files: log_mini.cvs and tf_mini.csv)

## Spotify music sessions dataset

The public part of the dataset consists of roughly 130 million listening sessions with associated user interactions on the Spotify service. In total, users interacted with almost 4 million tracks during these sessions, and the dataset includes acoustic features and metadata for all of these tracks.

Detailed description of the dataset can be found here:
https://drive.google.com/file/d/1BELTuH4nBeyHna5EAGzJv-HWHKrbxPsf/view?usp=sharing



In [2]:
!gdown --folder https://drive.google.com/drive/folders/10LGZMgXRuz2qPr_QDbYdVVlKEcnQ25YL?usp=sharing -O ./

Retrieving folder list
Processing file 1_jFxa2s7ONQ7nrhqFc1lARP6WChs2T4e data1.txt
Processing file 1rEJFi98sLUTYj50Fh8ff33A7NKOoT87c hmdata.zip
Processing file 1D3RyRBKBaw0KbLxpJUQDjVIt5CLpE2n9 log_mini.csv
Processing file 1BELTuH4nBeyHna5EAGzJv-HWHKrbxPsf Spotify-dataset-description.pdf
Processing file 1Z8N0Xf9M34PuN4V5G5uAxasvOCm6TVp4 tf_mini.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1_jFxa2s7ONQ7nrhqFc1lARP6WChs2T4e
To: /content/data/data1.txt
100% 158M/158M [00:01<00:00, 89.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rEJFi98sLUTYj50Fh8ff33A7NKOoT87c
To: /content/data/hmdata.zip
100% 773M/773M [00:09<00:00, 78.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D3RyRBKBaw0KbLxpJUQDjVIt5CLpE2n9
To: /content/data/log_mini.csv
100% 28.9M/28.9M [00:00<00:00, 120MB/s]
Downloading...
From: https://drive.google.com/uc?id=1BELTuH4nBeyHna5EAGzJv-HWHKrbxPs

In [3]:
log = pd.read_csv("data/log_mini.csv")
tracks = pd.read_csv("data/tf_mini.csv")

log.shape, tracks.shape

((167880, 21), (50704, 30))

In [4]:
log.head(2)

,session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,...,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
0,0_00006f66-33e5-4de7-a324-2d18e439fc1e,1,20,t_0479f24c-27d2-46d6-a00c-7ec928f2b539,False,False,False,True,0,0,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone
1,0_00006f66-33e5-4de7-a324-2d18e439fc1e,2,20,t_9099cd7b-c238-47b7-9381-f23f2c1d1043,False,False,False,True,0,1,...,0,0,0,True,16,2018-07-15,True,editorial_playlist,trackdone,trackdone


In [5]:
tracks.head(2)

,track_id,duration,release_year,us_popularity_estimate,acousticness,beat_strength,bounciness,danceability,dyn_range_mean,energy,...,time_signature,valence,acoustic_vector_0,acoustic_vector_1,acoustic_vector_2,acoustic_vector_3,acoustic_vector_4,acoustic_vector_5,acoustic_vector_6,acoustic_vector_7
0,t_a540e552-16d4-42f8-a185-232bd650ea7d,109.706673,1950,99.975414,0.458040,0.519497,0.504949,0.399767,7.511880,0.817709,...,4,0.935512,-0.033284,-0.411896,-0.02858,0.349438,0.832467,-0.213871,-0.299464,-0.675907
1,t_67965da0-132b-4b1e-8a69-0ef99b32287c,187.693329,1950,99.969430,0.916272,0.419223,0.545530,0.491235,9.098376,0.154258,...,3,0.359675,0.145703,-0.850372,0.12386,0.746904,0.371803,-0.420558,-0.213120,-0.525795


### Traditional metrics: ndcg@k

We will use the log dataframe as the main dataframe for evaluation of metrics. The skip_1 flag can be used as a relevance signal -- if the user found the recommendation relevant, skip_1 = False. With this relevance signal, we can compute simple ndcg metrics -- one for each session and then averaged across all sessions. This will serve as a base metric for comparison.

Note: the ranking logic here is assumed to be the production ranker, i.e. sorting by session_position gives the exact order of tracks the Spotify ranker presented to the user.

Lets compute simple skip rate and ndcg metric for the production ranker:

In [6]:
def skip_rate(sort_by=['session_position'], topk=10):
    return log.sort_values(by=sort_by).groupby("session_id").apply(lambda x: np.mean(x['skip_1'].values[:topk])).mean()

skip_rate(topk=10)

0.40894

In [8]:
def get_ndcg(df):
    true_relevance = np.asarray(1-np.asarray(df['skip_1']*1.0))
    ranker_scores = np.asarray(1/(np.asarray(range(len(df)))+1)) # approximate the ranker scores using the session position
    return ndcg_score([true_relevance], [ranker_scores])

def ndcg_metric(sort_by=['session_position'], topk=10):
    return log.groupby('session_id').progress_apply(lambda x: x.nsmallest(topk,sort_by)).reset_index(drop = True)\
                .groupby("session_id").apply(get_ndcg).mean()

ndcg_metric(topk=10)

100%|██████████| 10000/10000 [00:27<00:00, 365.64it/s] 


0.8330266041453142

## Goals for this week:

Implement a few behavioral metrics and compare their correlations. We will implement the following three metrics:
1. *Time to first skip:* how long did it take for the user to get the first bad recommendation, i.e. a recommendation they skipped. Since we can't easily calculate time, we can use number of songs as a proxy and compute the metric as number of songs it took for the first skip.

2. *Sustained dissatisfaction:* we assume that the user is dissatisfied in a sustained manner if they skip 3 songs consecutively.

3. *Session coherence:* we define coherence as how similar the recommended musical tracks are. We can use the acoustic_vector of the music tracks to calculate the similarity.

In [9]:
# implement session metric 1: time to first skip (number of songs to first skip)

def find_skip_1_ind(x, topk=20):
    x = x[:topk]
    inds = np.where(x==True)[0]
    if len(inds) == 0:
        return len(x) # no skips detected
    else:
        return inds[0]+1 # since index 0 is song 1, etc.

def time_to_first_skip(sort_by=['session_position'], topk=20):
    return log.sort_values(by=sort_by).groupby("session_id")['skip_1'].progress_apply(partial(find_skip_1_ind, topk=topk)).mean()

time_to_first_skip(topk=20)

100%|██████████| 10000/10000 [00:01<00:00, 5440.80it/s]


4.6348

In [10]:
# implement session metric 2: sustained dissatisfaction: proportions of sessions with 3 consecutive skips

def find_3_consecutive_skips(x, topk=20):
    count = 0
    for skip in x[:topk]:
        if skip == True:
            count += 1
        else:
            count = 0
        if count == 3:
            return 1
    return 0

def three_consecutive_skips(sort_by=['session_position'], topk=20):
    return log.sort_values(by=sort_by).groupby("session_id")['skip_1'].progress_apply(partial(find_3_consecutive_skips, topk=topk)).mean()

three_consecutive_skips(topk=20)

100%|██████████| 10000/10000 [00:00<00:00, 14818.44it/s]


0.6543

In [11]:
# implement session metric 3: session coherence: average similarity between the top recommended tracks

track2acoustic = {}
for idx, row in tqdm(tracks.iterrows(), total=len(tracks)):
    track2acoustic[row['track_id']] = np.array([row[f'acoustic_vector_{ind}'] for ind in range(8)])

def pairwise_cosine_sim(x, topk=20):
    a = np.array([track2acoustic[tid] for tid in x.values[:topk]])
    a /= np.linalg.norm(a, axis=1).reshape(-1,1)
    sims = a.dot(a.T)
    sims = np.triu(sims, k=1).flatten()
    inds = np.where(sims != 0)[0]
    return sims[inds].mean()

def session_coherence(sort_by=['session_position'], topk=20):
    return log.sort_values(by=sort_by).groupby("session_id")['track_id_clean'].progress_apply(partial(pairwise_cosine_sim, topk=topk)).mean()

session_coherence(topk=20)

100%|██████████| 10000/10000 [00:01<00:00, 7715.81it/s]


0.8097159162012062

Once these metrics are implemented, compute these metrics for topk=5 and topk=10 compare their estimates for the production ranker as a correlation plot. Please note which metrics are correlated with ndcg metric.

**Additional goal:**
Implement another simple ranking logic, and compare the performance of both the production ranker and new ranking policy on the ndcg and three behavioral metrics.
A simple ranking policy could include sortby track popularity, or sort by danceability score for the track.

Please report the performance of these rankers on all four metrics.

In [12]:
skip_rate(topk=5), ndcg_metric(topk=5), time_to_first_skip(topk=5), three_consecutive_skips(topk=5), session_coherence(topk=5)

100%|██████████| 10000/10000 [00:01<00:00, 9102.60it/s]


(0.3775, 0.8235700606177565, 3.0775, 0.2964, 0.8234136236006494)

In [13]:
skip_rate(topk=10), ndcg_metric(topk=10), time_to_first_skip(topk=10), three_consecutive_skips(topk=10), session_coherence(topk=10)

100%|██████████| 10000/10000 [00:01<00:00, 8845.67it/s]


(0.40894, 0.8330266041453142, 4.0886, 0.5238, 0.8174122151803728)

In [14]:
tracks['us_popularity_estimate_rev'] = tracks['us_popularity_estimate'].max() - tracks['us_popularity_estimate']
track2pop = dict(zip(tracks['track_id'], tracks['us_popularity_estimate_rev']))
log['us_popularity_estimate_rev'] = log['track_id_clean'].map(track2pop)

(skip_rate(sort_by=['us_popularity_estimate_rev'], topk=10), 
 ndcg_metric(sort_by=['us_popularity_estimate_rev'], topk=10), 
 time_to_first_skip(sort_by=['us_popularity_estimate_rev'], topk=10), 
 three_consecutive_skips(sort_by=['us_popularity_estimate_rev'], topk=10), 
 session_coherence(sort_by=['us_popularity_estimate_rev'], topk=10))

100%|██████████| 10000/10000 [00:01<00:00, 8725.25it/s]


(0.39834, 0.8132311534216118, 3.5449, 0.3799, 0.8324175261420991)

# Part B: Off Policy Evaluation

We log listener behavior based on the recommendations that the production recommender serves to the listener. Using this data to assess any new recommender system, however, can present challenges – the production recommender and the new recommender can drastically differ in the results that they display to the user. For example, maybe the new recommender presents a lot of niche content, while the production recommender presents a lot of popular options. This can be an issue when evaluating a new recommender – If you don’t have any feedback on a recommendation because you never presented it to a user, how can you evaluate whether it’s a good recommendation?
If you have a new policy to test that’s very similar to your old approach, then this won’t be an issue, and it’ll be easy to test! However, if the policy is very different, then you’ll need to collect special logged data.

In this part of the project, we will simulate a recommendation policy and leverage counterfactual estimators as metrics to compare performance.


Lets first begin by generating a few users and products. For ease of simulation, we assume users derive equal satisfaction from each item.

In [15]:
users = np.array(["user1", "user2", "user3"])

products = np.array(
    [
        "product_a",
        "product_b",
        "product_c",
        "product_d",
        "product_e",
        "product_f",
        "product_g",
    ]
)

satisfaction = {
    "product_a": 100,
    "product_b": 150,
    "product_c": 100,
    "product_d": 200,
    "product_e": 500,
    "product_f": 120,
    "product_g": 160,
}

Lets also implement whether a given user will accept a given recommendation or not. Once done, we can implement a target policy that makes recommendations.

In [16]:
def will_purchase(user, product):
    if user == "user1" and (
        product == "product_a" or product == "product_b" or product == "product_c"
    ):
        return True
    elif user == "user2" and (product == "product_d" or product == "product_e"):
        return True
    elif user == "user3" and (product == "product_f" or product == "product_g"):
        return True
    else:
        return False


def choose_user():
    return np.random.choice(users, size=1)


def logging_policy():
    return np.random.choice(products, size=1), 1 / len(products)


class TargetPolicy:
    def __init__(self):
        self.user_probs = {
            "user1": np.array([0.1, 0.1, 0.2, 0.1, 0.15, 0.15, 0.20]),
            "user2": np.array([0.1, 0.10, 0.05, 0.25, 0.3, 0.1, 0.1]),
            "user3": np.array([0.06, 0.06, 0.3, 0.06, 0.06, 0.4, 0.06]),
        }

        for user, probs in self.user_probs.items():
            assert probs.sum() == 1
            assert len(probs) == len(products)

    def recommend(self, user):
        user_prob = self.user_probs[user]
        product = np.random.choice(products, size=1, p=user_prob)
        product_idx = np.where(products == product)
        prob = user_prob[product_idx]

        return product, prob

    def get_prob(self, user, product):
        user_prob = self.user_probs[user]
        product_idx = np.where(products == product)
        product_prob = user_prob[product_idx]

        return product_prob

Having defined all key components of the dataset generation, lets create logged data that we can finally use for evaluation purposes:

In [17]:
def compute_satisfaction(user, product):
    if will_purchase(user, product):
        return satisfaction[product.item()]
    else:
        return 0


def create_logs(n=1000):
    logs = []
    target_policy = TargetPolicy()

    for _ in range(n):
        user = choose_user()

        logging_product, logging_prob = logging_policy()
        model_prob = target_policy.get_prob(user.item(), logging_product)

        target_product, _ = target_policy.recommend(user.item())

        logging_satisfaction = compute_satisfaction(user, logging_product)
        target_satisfaction = compute_satisfaction(user, target_product)

        log = OrderedDict(
            {
                "user_features": user.item(),
                "item_placed": logging_product.item(),
                "item_prob": logging_prob,
                "item_satisfaction": logging_satisfaction,
                "model_prob": model_prob.item(),
                "ab_test_satisfaction": target_satisfaction,
            }
        )

        logs.append(log)

    return pd.DataFrame(logs)

Here is what ur logged data now looks like:

In [18]:
logs = create_logs(n=1000)
logs.head(5)

,user_features,item_placed,item_prob,item_satisfaction,model_prob,ab_test_satisfaction
0,user2,product_a,0.142857,0,0.10,500
1,user1,product_a,0.142857,100,0.10,100
2,user2,product_g,0.142857,0,0.10,0
3,user1,product_d,0.142857,0,0.10,0
4,user1,product_e,0.142857,0,0.15,0


In [19]:
logs[["user_features", "item_placed", "item_prob", "item_satisfaction"]].sample(n=10)

,user_features,item_placed,item_prob,item_satisfaction
859,user2,product_c,0.142857,0
936,user3,product_e,0.142857,0
869,user3,product_b,0.142857,0
191,user1,product_a,0.142857,100
20,user3,product_b,0.142857,0
148,user1,product_e,0.142857,0
34,user2,product_g,0.142857,0
870,user3,product_c,0.142857,0
470,user1,product_b,0.142857,150
529,user1,product_b,0.142857,150


With all the dataset ready, lets compute the mean rewards (satisfaction) for the logging/production policy and the target policy:

In [20]:
%%time
sim = create_logs(n=100000)
logging_policy = sim["item_satisfaction"].mean()
target_policy = sim["ab_test_satisfaction"].mean()

print(f"Expected reward from logging policy: {logging_policy: .2f}")
print(f"Expected reward from target policy: {target_policy: .2f}")

Expected reward from logging policy:  63.27
Expected reward from target policy:  100.43
CPU times: user 12.8 s, sys: 698 ms, total: 13.5 s
Wall time: 12.8 s


Now lets implement the IPS estimator:

In [23]:
def compute_ips(df):
    assert {"model_prob", "item_prob", "item_satisfaction"}.issubset(df.columns)
    return (df["model_prob"] / df["item_prob"] * df["item_satisfaction"]).mean()

In [24]:
ips_est = compute_ips(logs)
ips_est

102.60179999999998

Computing the IPS estimator on our 1,000 entry log gives an average revenue of 109.34 (very close to the true performance of 100.99) compared with the average revenue of the logging policy of 63.36. Therefore, we should be confident to deploy our target policy to production and do an AB test comparing it with the logging policy as a final validation.

## Goals for this part of project:

Finish the implementation of two additional off-policy estimators:
1. Capped IPS
2. Normalized Capped Importance Sampling (NCIS)

Feel free to try different capping thresholds, and compare the reward and standard deviations of these estimators with the IPS estimator and mean reward.


In [39]:
def compute_capped_ips(df, cap=1000):
    return (np.minimum(logs['model_prob'] / logs['item_prob'], cap) * df["item_satisfaction"]).mean()

compute_capped_ips(logs, cap=1000), compute_capped_ips(logs, cap=1)

(102.60179999999998, 55.9938)

In [40]:
def compute_ncis(df, cap=1000):
    capped_probs = np.minimum(df['model_prob'] / df['item_prob'], cap)
    return (capped_probs * df["item_satisfaction"]).mean() / capped_probs.mean()

compute_ncis(logs, cap=1000), compute_ncis(logs, cap=1)

(102.04260651629072, 77.58275255289374)